# LLM Registry Testing (Stage 3)

## Objective
Validate model alias registry behavior independently of full ATS flows.

## Prerequisites
1. `config/model_aliases.yaml` exists.
2. Optional live calls require provider credentials (for OpenAI-backed aliases).


In [ ]:
import os
from pathlib import Path

from notebooks._utils import print_checkpoint, project_root
from src.llm.factory import build_default_llm_client
from src.llm.registry import ModelAliasRegistry

ROOT = project_root()
CONFIG_PATH = ROOT / "config" / "model_aliases.yaml"
assert CONFIG_PATH.exists(), f"Missing config: {CONFIG_PATH}"

registry = ModelAliasRegistry(CONFIG_PATH)
print("Loaded alias config:", CONFIG_PATH)
print_checkpoint("setup complete")


## 1) Alias load and expected defaults


In [ ]:
expected_aliases = [
    "summarizer_default",
    "extractor_default",
    "explainer_default",
    "embedding_default",
]

for alias in expected_aliases:
    found = registry.get(alias)
    print(alias, "->", found.model)

assert registry.get("summarizer_default").model, "summarizer_default missing model"
assert registry.get("embedding_default").model, "embedding_default missing model"
print_checkpoint("alias resolution assertions passed")


## 2) Unknown alias error path


In [ ]:
try:
    registry.get("does_not_exist")
    raise AssertionError("Expected KeyError for unknown alias")
except KeyError as exc:
    print("Caught expected KeyError:", str(exc))

print_checkpoint("unknown alias assertion passed")


## 3) Optional live call checks (credential-gated)

Live calls are optional and skipped if credentials are unavailable.


In [ ]:
has_openai_key = bool(os.getenv("OPENAI_API_KEY"))

if not has_openai_key:
    print("Skipping live LLM checks (OPENAI_API_KEY not set)")
else:
    from pydantic import BaseModel

    class PingSchema(BaseModel):
        label: str

    client = build_default_llm_client()

    result = client.generate_structured(
        prompt="Return JSON with one field: label='ok'",
        schema=PingSchema,
        model_alias="summarizer_default",
        max_tokens=50,
    )

    assert result.label.lower() == "ok"

    vectors = client.embed(
        texts=["hello world", "resume parsing"],
        embedding_model_alias="embedding_default",
    )
    assert len(vectors) == 2
    assert all(isinstance(v, list) and v for v in vectors)

    print("Live checks passed")

print_checkpoint("optional live checks complete")


## Summary (fill after run)

- Alias loading: pass/fail
- Unknown alias path: pass/fail
- Optional live checks run: yes/no


## Next actions

1. If alias loading fails, fix `config/model_aliases.yaml` first.
2. If live calls fail, validate provider credentials and model access.
3. Mirror repeatable failures as unit tests in `tests/test_llm_registry.py`.
